In [32]:
import pandas as pd
import random
import json

In [33]:
#Sample data very similar to i2b2 2010 NER Dataset as this want available for download
data = {
    "Sentence": [
        "Mr. John Doe presents with shortness of breath and elevated blood pressure.",
        "Patient reports experiencing fatigue and dizziness for approximately 1 week.",
        "Initiated Lisinopril 10mg daily for hypertension management.",
        "Ordered complete blood count and ECG to rule out cardiac abnormalities.",
        "Dr. Patel advised lifestyle modifications and scheduled a stress test in 10 days.",
        "No prior history of myocardial infarction or cerebrovascular events documented.",
        "Complains of lower back pain exacerbated by movement over the last 3 days.",
        "Prescribed Ibuprofen 400mg TID along with physical therapy sessions.",
        "Scheduled follow-up appointment with neurologist next Thursday.",
        "Family history includes Type 2 diabetes and chronic kidney disease."
    ],
    "Entities": [
        ["problem:shortness of breath", "problem:elevated blood pressure"],
        ["problem:fatigue", "problem:dizziness", "duration:1 week"],
        ["treatment:Lisinopril 10mg", "problem:hypertension"],
        ["test:complete blood count", "test:ECG", "problem:cardiac abnormalities"],
        ["doctor:Dr. Patel", "treatment:lifestyle modifications", "test:stress test", "duration:10 days"],
        ["problem:myocardial infarction", "problem:cerebrovascular events"],
        ["problem:lower back pain", "duration:3 days"],
        ["treatment:Ibuprofen 400mg", "treatment:physical therapy"],
        ["test:follow-up appointment", "doctor:neurologist", "duration:next Thursday"],
        ["problem:Type 2 diabetes", "problem:chronic kidney disease", "entity_type:family history"]
    ]
}
# Convert to DataFrame for easier visualization
df = pd.DataFrame(data)
df.head(3)

,Sentence,Entities
0,Mr. John Doe presents with shortness of breath...,"[problem:shortness of breath, problem:elevated..."
1,Patient reports experiencing fatigue and dizzi...,"[problem:fatigue, problem:dizziness, duration:..."
2,Initiated Lisinopril 10mg daily for hypertensi...,"[treatment:Lisinopril 10mg, problem:hypertension]"


In [34]:
# Simulated Agent Invocation

def supervisor_agent(intent: str, payload: dict):
    if intent == "check_in":
        return patient_checkin_agent(payload)
    elif intent == "start_consultation":
        return soap_compiler_agent(payload)
    else:
        raise ValueError("Unknown intent")

In [35]:
# Simulated Tools and Models
def preprocess(data):
    return {"processed": data}

def code_interpreter(data):
    return preprocess(data)

In [36]:
# Common Tool

def open_search_vector_query(query_text):
    return f"Results for: {query_text}"

def query_knowledge_base(query_text):
    return open_search_vector_query(query_text)

class Model:
    def predict(self, input_features):
        return random.uniform(15, 120)

class NER:
    def extract(self, input_features):
        idx = random.randint(0, len(data["Entities"]) - 1)
        return data["Entities"][idx]

class SOAPClassifier:
    def predict(self, input_features):
        return {
            "subjective": "Patient complains of headache.",
            "objective": "Vitals normal.",
            "assessment": "Possible migraine.",
            "plan": "Prescribe rest and monitor."
        }

In [37]:
#combining tools

lstm_model = Model()
xgboost_model = Model()
clinical_ner_model = NER()
soap_classifier = SOAPClassifier()

In [38]:
#memory implementation via Action Group

# Agent Functions (from user code)
short_term_memory = {}
long_term_memory = {}

def action_group_execute(model_type, input_features):
    if model_type == "wait_time":
        return lstm_model.predict(input_features), xgboost_model.predict(input_features)
    elif model_type == "soap_note":
        return {
            "entities": clinical_ner_model.extract(input_features),
            "soap_structure": soap_classifier.predict(input_features)
        }
    else:
        raise ValueError("Unsupported model type")

def update_memory(patient_id, value, memory_type="short"):
    if memory_type == "short":
        short_term_memory[patient_id] = value
    else:
        long_term_memory.setdefault(patient_id, []).append(value)

In [39]:
def patient_checkin_agent(payload):
    data = code_interpreter(payload["qms_event"])
    context = query_knowledge_base("clinic queue patterns")
        prediction = action_group_execute("wait_time", data)
    update_memory(payload["patient_id"], prediction, memory_type="short")
    return {
        "api": "/get_estimated_wait_time",
        "response": {
            "patient_id": payload["patient_id"],
            "estimated_wait": round(sum(prediction) / 2, 2),
            "context": context
        }
    }

In [40]:
def soap_compiler_agent(payload):
    transcript = code_interpreter(payload["transcript"])
    history = query_knowledge_base(f"guidelines for {payload['chief_complaint']}")
    result = action_group_execute("soap_note", transcript)
    update_memory(payload["patient_id"], result, memory_type="long")
    return {
        "api": "/generate_soap_note",
        "response": {
            "patient_id": payload["patient_id"],
            "soap_note": result,
            "context": history
        }
    }

In [41]:
checkin_result = supervisor_agent("check_in", {"patient_id": "P001", "qms_event": {"queue_length": 5}})
soap_result = supervisor_agent("start_consultation", {
    "patient_id": "P001", 
    "transcript": "Patient complains of persistent headache and fatigue.",
    "chief_complaint": "headache"
})

checkin_result, soap_result

({'api': '/get_estimated_wait_time',
  'response': {'patient_id': 'P001',
   'estimated_wait': 80.8,
   'context': 'Results for: clinic queue patterns'}},
 {'api': '/generate_soap_note',
  'response': {'patient_id': 'P001',
   'soap_note': {'entities': ['problem:fatigue',
     'problem:dizziness',
     'duration:1 week'],
    'soap_structure': {'subjective': 'Patient complains of headache.',
     'objective': 'Vitals normal.',
     'assessment': 'Possible migraine.',
     'plan': 'Prescribe rest and monitor.'}},
   'context': 'Results for: guidelines for headache'}})

In [42]:
# Check-In Result
print("Check-In Agent Result:")
print(f"API: {checkin_result['api']}")
print(f"Patient ID: {checkin_result['response']['patient_id']}")
print(f"Estimated Wait Time: {checkin_result['response']['estimated_wait']} minutes")
print(f"Context Info: {checkin_result['response']['context']}\n")

# SOAP Note Result
print("SOAP Compiler Agent Result:")
print(f"API: {soap_result['api']}")
print(f"Patient ID: {soap_result['response']['patient_id']}")
print("Extracted Entities:", ", ".join(soap_result['response']['soap_note']['entities']))
print("SOAP Note Structure:")
for section, text in soap_result['response']['soap_note']['soap_structure'].items():
    print(f"  {section.title()}: {text}")
print(f"Context Info: {soap_result['response']['context']}")

Check-In Agent Result:
API: /get_estimated_wait_time
Patient ID: P001
Estimated Wait Time: 80.8 minutes
Context Info: Results for: clinic queue patterns

SOAP Compiler Agent Result:
API: /generate_soap_note
Patient ID: P001
Extracted Entities: problem:fatigue, problem:dizziness, duration:1 week
SOAP Note Structure:
  Subjective: Patient complains of headache.
  Objective: Vitals normal.
  Assessment: Possible migraine.
  Plan: Prescribe rest and monitor.
Context Info: Results for: guidelines for headache


In [43]:
# Simulated clinician update to diagnosis
soap_result['response']['soap_note']['soap_structure']['assessment'] = "Confirmed tension-type headache"

# Update long-term memory
long_term_memory.setdefault("P001", []).append(soap_result['response'])